pandas: 데이터를 쉽고 유연하게 처리하기 위한 라이브러리입니다.

.smiles2fing: SMILES 문자열들을 입력으로 받아 fingerprint로 바꿔주는 함수를 포함하고 있는 파이썬 파일입니다.
- smiles2fing: 변환하지 못한 문자열들의 인덱스와, 그를 제외하고 변환에 성공해서 얻은 fingerprint들을 반환하는 함수입니다.

In [ ]:
import pandas as pd
from .smiles2fing import smiles2fing

load_data: 설정한 경로에 해당하는 엑셀 파일을 불러오고, 전처리와 타겟 변수를 반환하는 함수입니다.


함수의 동작 과정은 아래와 같습니다.
1. 함수의 입력으로 넣어준 경로(path, tg_num, inhale_type이 함께 경로를 구성)에 있는 엑셀 파일을 pandas 데이터프레임으로 변환하고, 이를 df 변수에 저장합니다.
2. smiles2fing 함수(smiles2fing.ipynb 참조)를 통해 SMILES 형식의 데이터를 fingerprint 형식으로 바꿔주는 등의 전처리 과정을 수행합니다.
    - 위 함수의 반환값으로, fingerprint로의 변환에 실패한 레코드들의 인덱스와 fingerprint 형식으로 변환된 이후의 데이터프레임을 반환합니다.

3. df에서 fingerprint로의 변환에 실패한 친구들을 제외한 레코드들의 category열의 값을 가져와, y에 저장합니다.
4. 결과적으로, fingerprint로의 변환에 성공한 레코드들만의 변환 이후 데이터프레임과 category열의 값들을 반환합니다.


In [ ]:
def load_data(path: str, tg_num: int, inhale_type: str):
    try:
        df = pd.read_excel(f'{path}tg{tg_num}_{inhale_type}.xlsx')
    except:
        df = pd.read_excel(f'{path}/tg{tg_num}_{inhale_type}.xlsx')
    
    drop_idx, fingerprints = smiles2fing(df.SMILES)
    
    y = df.category.drop(drop_idx).reset_index(drop = True)
    
    return fingerprints, y

multiclass2binary: catetory의 종류를 3개에서 2개로 줄여줍니다.
- category열의 값으로 가능한 값들은 [0, 1, 2]입니다.
- multi-classification이 아닌 binary classification을 위해서 category의 종류를 2개로 줄여줍니다.


함수의 동작은 아래와 같습니다.
1. 데이터의 tg_num이 403인 경우에는 아래와 같이 변환합니다.
    1. y값이 0 또는 1이면, 1로 변환합니다.
    2. y값이 2 이면, 0으로 변환합니다.

2. 데이터의 tg_num이 403이 아닌 경우에는 아래와 같이 변환합니다.
    1. y값이 0 이면, 1로 변환합니다.
    2. y값이 1 또는 2이면, 0으로 변환합니다.

3. 변환시킨 데이터 프레임을 반환합니다.

In [ ]:
def multiclass2binary(y, tg_num: int):
    bin_y = y.copy()
    
    if tg_num == 403:
        bin_y[y<2] = 1
        bin_y[y>=2] = 0
    
    else:
        bin_y[y<1] = 1
        bin_y[y>=1] = 0
    
    return bin_y

사용되지 않는 함수입니다.

In [ ]:
def load_pred_data():
    df_tmp = pd.read_excel('pred_data.xlsx').drop_duplicates(subset = ('PREFERRED_NAME', 'SMILES'))
    
    # try:
    #     df = pd.read_excel(f'{path}/pred_data.xlsx')
    # except: 
    #     df = pd.read_excel(f'{path}pred_data.xlsx')
    
    df = df_tmp[df_tmp['SMILES'].notna()].reset_index(drop = True)
    
    drop_idx, fingerprints = smiles2fing(df.SMILES)
    df = df.drop(drop_idx).reset_index(drop = True)
    
    return fingerprints, df, df_tmp